<a href="https://colab.research.google.com/github/Aakash3101/Deep-Learning/blob/master/ML/Deep_learning/pytorch_tutorials/cifar10/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Transfer Learning using ResNet and other optimizations for CNNs**

In [1]:
import os
import torch
import torchvision
import tarfile
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as tt
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: agg


In [2]:
project_name = '05b-cifar10-resnet'

## **Preparing the Data**

In [3]:
# Download the dataset
dataset_url = "https://s3.amazonaws.com/fast-ai-sample/cifar10.tgz"
download_url(dataset_url, '.')

# Extract from archive
with tarfile.open('./cifar10.tgz', 'r:gz') as tar:
    tar.extractall(path='./data')

# Look into the data directory
data_dir = './data/cifar10'
print(os.listdir(data_dir))
classes = os.listdir(data_dir + "/train")
print(classes)

['train', 'labels.txt', 'test']
['dog', 'deer', 'cat', 'automobile', 'ship', 'frog', 'airplane', 'truck', 'bird', 'horse']


In [4]:
len(os.listdir(data_dir + '/train/horse'))

5000

There are a few important changes we'll make while creating the PyTorch datasets:

1. **Use test set for validation:** Instead of setting aside a fraction (e.g. 10%) of the data from the training set for validation, we'll simply use the test set as our validation set **(Only when you have labels for your test set)**. This just gives a little more data to train with. In general, once you have picked the best model architecture & hyperparameters using a fixed validation set, it is a good idea to retrain the same model on the entire dataset just to give it a small final boost in performance

2. **Channel-wise data normalization:** We will normalize the image tensors by subtracting the mean and dividing by the standard deviation across each channel. As a result, the mean of the data across each channel is 0, and standard deviation is 1. Normalizing the data prevents the values from any one channel from disproportionately affecting the losses and gradients while training, simply by having a higher or wider range of values than others.

3. **Randomized data augmentations:** We will apply randomly chosen transformations while loading images from the training dataset. Specifically, we will pad each image by 4 pixels, and then take a random crop of size 32x32 pixels, and then flip the image horizontally with a 50% probability. Since the transformation will be applied randomly and dynamically each time a particular image is loaded, the model sees slightly different images in each epoch of training, which allows it generalize better.

In [5]:
# Data transforms (normalization and data augmentation)

# Mean and Standard Deviation for each color channel
stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

train_tfms = tt.Compose([tt.RandomCrop(32, padding=4, padding_mode='reflect'),
                         tt.RandomHorizontalFlip(),
                         tt.ToTensor(),
                         tt.Normalize(*stats, inplace=True)])

valid_tfms = tt.Compose([tt.ToTensor(), tt.Normalize(*stats)])

In [6]:
# PyTorch datasets

train_ds = ImageFolder(data_dir + '/train', train_tfms)
val_ds = ImageFolder(data_dir + '/test', valid_tfms)

In [ ]:
36:15

## **Creating the model**

### Observation Table
---
Model 1:

```
```

---
| Model S.no | I (Lr, Epochs) | II (Lr, Epochs) | optimizer | val_acc | test_acc |
| --- | --- | --- | --- | --- | --- |
| 1 | (0.1, 20) | (0.01, 10) | SGD | 55.37 | 54.74 |
| 1 | (0.2, 20) | (0.1, 10) | SGD | 49.40 | 48.86 |
| 1 | (0.5, 20) | (0.1, 10) | SGD | 48.87 | 50.66 |
| 1 | (0.1, 30) | (0.01, 10) | SGD | 55.66 | 55.31 |
| 1 | (0.001, 20) | (0.0001, 10) | Adam | 53.04 | 53.34 |
| 1 | (0.0001, 20) | (1e-5, 10) | Adam | 51.62 | 52.17 |

---

In [ ]:
torch.cuda.empty_cache()